In [1]:
!locale -a

C
C.utf8
en_US.utf8
POSIX


In [2]:
!export LC_ALL=en_US.UTF-8

In [3]:
!pip install autotrain-advanced
!pip install huggingface_hub

In [4]:
import autotrain
import huggingface_hub

print(autotrain.__version__)
print(huggingface_hub.__version__)

0.6.88
0.20.3


In [5]:
!pip install autotrain-advanced
!pip install huggingface_hub

In [6]:
push_to_hub = False
hf_token = "hf_token"
repo_id = "repo_id"

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
model = AutoModelForCausalLM.from_pretrained(repo_id, token = hf_token)

In [ ]:

from transformers import AutoModelForCausalLM, AutoTokenizer

model_path = "mistral-7b-instruct-v0.1.Q4_K_M.gguf"

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    device_map="auto",
    torch_dtype='auto'
).eval()

# Prompt content: "hi"
messages = [
    {"role": "user", "content": "hi"}
]

input_ids = tokenizer.apply_chat_template(conversation=messages, tokenize=True, add_generation_prompt=True, return_tensors='pt')
output_ids = model.generate(input_ids.to('cuda'))
response = tokenizer.decode(output_ids[0][input_ids.shape[1]:], skip_special_tokens=True)

# Model response: "Hello! How can I assist you today?"
print(response)


In [ ]:
!autotrain setup

# Loading Datasets

In [ ]:
from datasets import load_dataset

# Load the dataset
dataset = load_dataset("tatsu-lab/alpaca")
train = dataset['train']

# Loading our training data

In [ ]:
import pandas as pd

train1 = pd.read_csv('intent_alpaca_format.csv')
train2 = pd.read_csv('intent2_alpaca_format.csv')

train_combined = pd.concat([train1, train2], ignore_index=True)

from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("mistral-7b-instruct-v0.2.Q4_K_M.gguf")
model = AutoModelForSequenceClassification.from_pretrained("mistral-7b-instruct-v0.2.Q4_K_M.gguf", num_labels=len(set(train_combined.intent)))



In [ ]:
# Tokenize and encode the training data
encoded_data = tokenizer(train_combined.text.tolist(), truncation=True, padding=True)



In [ ]:
# Finetune the model
model.train_dataloader = encoded_data
model.train()

In [ ]:
project_name = 'TheBloke/Mistral-7B-Instruct-v0.2-GGUF'
model_name = 'mistral-7b-instruct-v0.2.Q4_K_M.gguf'

In [ ]:
push_to_hub = False
hf_token = "your_hf_token"
repo_id = "your_repo_id"

In [ ]:
learning_rate = 2e-4
num_epochs = 4
batch_size = 1
block_size = 1024
trainer = "sft"
warmup_ratio = 0.1
weight_decay = 0.01
gradient_accumulation = 4
use_fp16 = True
use_peft = True
use_int4 = True
lora_r = 16
lora_alpha = 32
lora_dropout = 0.045

In [ ]:
import os
os.environ["PROJECT_NAME"] = project_name
os.environ["MODEL_NAME"] = model_name
os.environ["PUSH_TO_HUB"] = str(push_to_hub)
os.environ["HF_TOKEN"] = hf_token
os.environ["REPO_ID"] = repo_id
os.environ["LEARNING_RATE"] = str(learning_rate)
os.environ["NUM_EPOCHS"] = str(num_epochs)
os.environ["BATCH_SIZE"] = str(batch_size)
os.environ["BLOCK_SIZE"] = str(block_size)
os.environ["WARMUP_RATIO"] = str(warmup_ratio)
os.environ["WEIGHT_DECAY"] = str(weight_decay)
os.environ["GRADIENT_ACCUMULATION"] = str(gradient_accumulation)
os.environ["USE_FP16"] = str(use_fp16)
os.environ["USE_PEFT"] = str(use_peft)
os.environ["USE_INT4"] = str(use_int4)
os.environ["LORA_R"] = str(lora_r)
os.environ["LORA_ALPHA"] = str(lora_alpha)
os.environ["LORA_DROPOUT"] = str(lora_dropout)

In [ ]:
import locale

In [ ]:
current_encoding = locale.getpreferredencoding()
print(f"Current locale encoding: {current_encoding}")

In [ ]:
!export LANG=en_US.UTF-8

In [ ]:
!autotrain llm \
--train \
--model ${MODEL_NAME} \
--project-name ${PROJECT_NAME} \
--data-path data/ \
--text-column text \
--lr ${LEARNING_RATE} \
--batch-size ${BATCH_SIZE} \
--epochs ${NUM_EPOCHS} \
--block-size ${BLOCK_SIZE} \
--warmup-ratio ${WARMUP_RATIO} \
--lora-r ${LORA_R} \
--lora-alpha ${LORA_ALPHA} \
--lora-dropout ${LORA_DROPOUT} \
--weight-decay ${WEIGHT_DECAY} \
--gradient-accumulation ${GRADIENT_ACCUMULATION} \
$( [[ "$USE_FP16" == "True" ]] && echo "--fp16" ) \
$( [[ "$USE_PEFT" == "True" ]] && echo "--use-peft" ) \
$( [[ "$USE_INT4" == "True" ]] && echo "--use-int4" ) \
$( [[ "$PUSH_TO_HUB" == "True" ]] && echo "--push-to-hub --token ${HF_TOKEN} --repo-id ${REPO_ID}" )

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_path = "mistral-7b-instruct-v0.1.Q4_K_M.gguf"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path)

In [ ]:
# learning_rate = 2e-4
# num_epochs = 4
# batch_size = 1
# block_size = 1024
# trainer = "sft"
# warmup_ratio = 0.1
# weight_decay = 0.01
# gradient_accumulation = 4
# use_fp16 = True
# use_peft = True
# use_int4 = True
# lora_r = 16
# lora_alpha = 32
# lora_dropout = 0.045

In [ ]:
input_text = "What are hair and beauty products there is?"
input_ids = tokenizer.encode(input_text, return_tensors="pt")
output = model.generate(input_ids)
predicted_text = tokenizer.decode(output[0], skip_special_tokens=False)
print(predicted_text)

In [ ]:
# !autotrain train llm --project-name my-project
# --model model \
# --project-name -ourphi \
# --data-path -data /content/  \
# # --text-column /text \
# --lr /2e-4 \
# --batch-size /2 \
# --epochs /3 \
# --block-size /1024, \
# --warmup-ratio /0.1 \
# --lora-r /16 \
# --lora-alpha /32 \
# --lora-dropout /0.05 \
# --weight-decay /0 \
# --gradient-accumulation /1 \
# --USE_FP16 == True, echo --fp16 \
# --USE_PEFT == True, echo --use-peft \
# --USE_INT4 == True, echo --use-int4 \
# --PUSH_TO_HUB == True, echo --push-to-hub --token 

In [ ]:
trainer.push_to_hub()

In [ ]:
model.push_to_hub("Gmannk/autotrain-ecomm")
tokenizer.push_to_hub("mistral-7b-instruct-v0.2.Q4_K_M.gguf")

## Run the finetuned model

In [ ]:
# generation_config = model.generation_config
# generation_config.max_new_tokens = 200
# generation_config.temperature = 0.7
# generation_config.top_p = 0.7
# generation_config.num_return_sequences = 1
# generation_config.pad_token_id = tokenizer.eos_token_id
# generation_config.eos_token_id = tokenizer.eos_token_id

In [ ]:
# input_text = "What are some electronic devices for beauty that amazon has?"
# input_ids = tokenizer.encode(input_text, return_tensors="pt")
# output = model.generate(input_ids)
# predicted_text = tokenizer.decode(output[0], skip_special_tokens=False)
# print(predicted_text)